<img src="https://whatcar.vn/media/2018/09/car-lot-940x470.jpg"/>

## Прогнозирование стоимости автомобиля по характеристикам
*Этот Ноутбук является Примером/Шаблоном (Baseline) к этому соревнованию и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.


> **baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline является хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

Помним, что по условию соревнования, нам нужно самостоятельно собрать обучающий датасет. В этом ноутбуке мы не будем рассматривать сбор данных. Предположим, что мы уже все собрали и просто подключили свой датасет через "Add Data", чтобы приступить к самому ML.

In [11]:
"""
from fake_useragent import UserAgent 
from bs4 import BeautifulSoup    
import requests  
import re
import time
import pandas as pd
import numpy as np
import itertools
from joblib import Parallel, delayed
from tqdm.notebook import tqdm




# Функция по извлечению данных со страницы объявления в словарь data_dict
def parsing_page_one_ad(url):

    response = requests.get(url, headers={'User-Agent': UserAgent().chrome})    
    response.encoding ='utf8'   
    
    # Теперь создадим объект BeautifulSoup, указывая html парсер    
    page = BeautifulSoup(response.text, 'html.parser')
    data_dict = {}

    data_dict['car_url'] = url
    data_dict['parsing_unixtime'] = int(time.time())


     # в разделах script ищем вхождение 'complectation":{"id"'
    for script in page.find_all("script"):
        if 'complectation":{"id"' in str(script):
            a = str(script)  # присваиваем a содержимое скрипта как строку
    # в a  ищем 'complectation":' и содержимое между {}. Отсекаем начало, оставляя только содержимое словаря
            data_dict['complectation_dict'] = re.search(r'complectation":{"id.*?}', a)[0][15:]

        if 'equipment":{' in str(script):
            a = str(script)  # присваиваем a содержимое скрипта как строку
        # в a  ищем 'equipment":' и содержимое между {}. Отсекаем начало, оставляя только содержимое словаря
            data_dict['equipment_dict'] = re.search(r'equipment":{.*?}', a)[0][11:]

        if '{"mileage":' in str(script):
            a = str(script)  # присваиваем a содержимое скрипта как строку
        # в a  ищем '{"mileage":' и содержимое между {}. Отсекаем начало, оставляя только содержимое словаря
            data_dict['mileage'] = re.search(r'"mileage":\d*', a)[0][10:]

        if '"model_info":' in str(script):
            a = str(script)  # присваиваем a содержимое скрипта как строку
            data_dict['model_info'] = re.search(r'"model_info":{.*?}', a)[0][13:]
            data_dict['model_name'] = re.search(r'model_info":{"code":".*?"', a)[0][20:].strip('"')

        if 'super_gen":{' in str(script):
            a = str(script)  # присваиваем a содержимое скрипта как строку
            data_dict['super_gen'] = re.search(r'super_gen":{.*?}', a)[0][11:] 

        if 'vendor":"' in str(script):
            a = str(script)  # присваиваем a содержимое скрипта как строку
            data_dict['vendor'] = re.search(r'vendor":".*?"', a)[0][9:].strip('"')


    for tag in page.find_all('div'):
        if tag.get("title") == "Идентификатор объявления":
            data_dict['sell_id'] = re.search(r'\d+', tag.text)[0]


    for tag in page.find_all("meta"):
        if tag.get("itemprop") == "bodyType":
            data_dict['bodyType'] = tag.get("content")

        if tag.get("itemprop") == "brand":
            data_dict['brand'] = tag.get("content")

        if tag.get("itemprop") == "color":
            data_dict['color'] = tag.get("content")

        if tag.get("itemprop") == "description":
            data_dict['description'] = tag.get("content")

        if tag.get("itemprop") == "engineDisplacement":
            data_dict['engineDisplacement'] = tag.get("content")

        if tag.get("itemprop") == "enginePower":
            data_dict['enginePower'] = tag.get("content")

        if tag.get("itemprop") == "fuelType":
            data_dict['fuelType'] = tag.get("content")

        if tag.get("itemprop") == "modelDate":
            data_dict['modelDate'] = tag.get("content")

        if tag.get("itemprop") == "name":
            data_dict['name'] = tag.get("content")

        if tag.get("itemprop") == "numberOfDoors":
            data_dict['numberOfDoors'] = tag.get("content")

        if tag.get("itemprop") == "price":
            data_dict['price'] = tag.get("content")

        if tag.get("itemprop") == "priceCurrency":
            data_dict['priceCurrency'] = tag.get("content")

        if tag.get("itemprop") == "productionDate":
            data_dict['productionDate'] = tag.get("content")

        if tag.get("itemprop") == "vehicleConfiguration":
            data_dict['vehicleConfiguration'] = tag.get("content")

        if tag.get("itemprop") == "vehicleTransmission":
            data_dict['vehicleTransmission'] = tag.get("content")


    span_CardInfoRow__cell = page.find_all('span', {'class': 'CardInfoRow__cell'})

    for i,tag in enumerate (span_CardInfoRow__cell):
        if tag.text == "Владельцы":
            data_dict['Владельцы'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел

        if tag.text == "Владение":
            data_dict['Владение'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел    

        if tag.text == "ПТС":
            data_dict['ПТС'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел    

        if tag.text == "Привод":
            data_dict['Привод'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел 

        if tag.text == "Руль":
            data_dict['Руль'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел

        if tag.text == "Состояние":
            data_dict['Состояние'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел 

        if tag.text == "Таможня":
            data_dict['Таможня'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел

    return data_dict

    
# Функция по созданию списка ссылок links_list  на объявления о продаже автомобилей  
def extraction_links(url):
    links_list =[] 
    response = requests.get(url, headers={'User-Agent': UserAgent().chrome})  
    response.encoding ='utf8'
    page = BeautifulSoup(response.text, 'html.parser') 
    links = page.find_all('a', class_='Link ListingItemTitle-module__link')
    
    for link in links:
        links_list.append(link.get("href"))
    return links_list
    
    
    
    
    
df = pd.DataFrame() # инициализируем итоговый датафрейм 

url_link_list = [] # список страниц по годам и номерам от 1 до 99

ranges = [range(2006, 2011), range(1, 100)] # указываем года и диапазон страниц, которые будем парсить 

# index[0] - year, index[1] - page 
for index in itertools.product(*ranges):
    # формируем ссылки страницы со списками объявлений
    на  = (f"https://auto.ru/moskva/cars/{index[0]}-year/all/?output_type=table&page={index[1]}")
    url_link_list.append(url_links) # заносим их в список
    
links_list = [] # список списков ссылок на объявления c одной страницы таблицы объявлений

#for url_links in url_link_list:
# извлекаем в links_list список ссылок на объявления
try:
        #links_list = extraction_links(url_links)
    links_list = Parallel(n_jobs = 2)(delayed(extraction_links)(url_links) for url_links in url_link_list)
except:
    pass 

ads_dict_list = [] # список словарей содержимого объявлений

for links in links_list:
    try:
        ads_dict_list = Parallel(n_jobs = 2)(delayed(parsing_page_one_ad)(ad_url) for ad_url in links)
    except:
        pass
    
    for ad in ads_dict_list:
        try:
            df = df.append(ad, ignore_index=True)
        except:
            pass

df.to_csv('_auto_ru_XXXX-XXXX.csv', encoding = 'utf-8', index=False) # записываем содержимое датафрейма в файл
"""

'\nfrom fake_useragent import UserAgent \nfrom bs4 import BeautifulSoup    \nimport requests  \nimport re\nimport time\nimport pandas as pd\nimport numpy as np\nimport itertools\nfrom joblib import Parallel, delayed\nfrom tqdm.notebook import tqdm\n\n\n\n\n# Функция по извлечению данных со страницы объявления в словарь data_dict\ndef parsing_page_one_ad(url):\n\n    response = requests.get(url, headers={\'User-Agent\': UserAgent().chrome})    \n    response.encoding =\'utf8\'   \n    \n    # Теперь создадим объект BeautifulSoup, указывая html парсер    \n    page = BeautifulSoup(response.text, \'html.parser\')\n    data_dict = {}\n\n    data_dict[\'car_url\'] = url\n    data_dict[\'parsing_unixtime\'] = int(time.time())\n\n\n     # в разделах script ищем вхождение \'complectation":{"id"\'\n    for script in page.find_all("script"):\n        if \'complectation":{"id"\' in str(script):\n            a = str(script)  # присваиваем a содержимое скрипта как строку\n    # в a  ищем \'complect

In [12]:
"""
import os
import pandas as pd
import numpy as np

df = pd.DataFrame()
list_file = os.listdir(path=".") # получение списка файлов в папке

for file in list_file:
    if '.csv' in file: # если у файла расширение .csv, то читаем его как DF
        df_temp = pd.read_csv(file)
        df = pd.concat([df, df_temp], ignore_index=True)
        
df = df.drop_duplicates(subset=['car_url']) # удаление дубликатов по URL объявления

df = df.replace('{}',  None) # замена пустых словарей на NaN
df['mileage'] = df['mileage'].apply(lambda x: int(x) if type(x) == str else x) # приведение 'mileage' к числовму виду
df = df.replace(np.nan,  None)

df.to_csv('_auto_ru_23012021.csv', encoding = 'utf-8', index=False)
"""

'\nimport os\nimport pandas as pd\nimport numpy as np\n\ndf = pd.DataFrame()\nlist_file = os.listdir(path=".") # получение списка файлов в папке\n\nfor file in list_file:\n    if \'.csv\' in file: # если у файла расширение .csv, то читаем его как DF\n        df_temp = pd.read_csv(file)\n        df = pd.concat([df, df_temp], ignore_index=True)\n        \ndf = df.drop_duplicates(subset=[\'car_url\']) # удаление дубликатов по URL объявления\n\ndf = df.replace(\'{}\',  None) # замена пустых словарей на NaN\ndf[\'mileage\'] = df[\'mileage\'].apply(lambda x: int(x) if type(x) == str else x) # приведение \'mileage\' к числовму виду\ndf = df.replace(np.nan,  None)\n\ndf.to_csv(\'_auto_ru_23012021.csv\', encoding = \'utf-8\', index=False)\n'

In [13]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [14]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

Python       : 3.7.9 (default, Aug 31 2020, 17:10:11) [MSC v.1916 64 bit (AMD64)]
Numpy        : 1.19.2


In [15]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [16]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [17]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

# Setup

In [18]:
VERSION    = 16
DIR_TRAIN  = '../input/parsing-all-moscow-auto-ru-09-09-2020/' # подключил к ноутбуку внешний датасет
#DIR_TRAIN  = '../input/auto-ru-moskva-23-01-2021/' # подключил к ноутбуку внешний датасет
DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%

# Data

In [19]:
#!ls '../input'

In [20]:
train = pd.read_csv('D:/Temp/all_auto_ru_09_09_2020.csv') # датасет для обучения модели
test = pd.read_csv('D:/Temp/test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

#train = pd.read_csv(DIR_TRAIN+'auto_ru_moskva_23_01_2021.csv') # датасет для обучения модели
#test = pd.read_csv(DIR_TEST+'test.csv')
#sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

In [21]:
train.head(5)

,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,...,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,price,start_date,hidden,model
0,Седан,AUDI,040001,бензин,1990.0,2.8 MT (174 л.с.) 4WD,4.0,1991,SEDAN MECHANICAL 2.8,MECHANICAL,...,LEFT,NaN,3.0,ORIGINAL,True,NaN,200000.0,2019-10-03T08:09:11Z,NaN,100
1,Седан,AUDI,EE1D19,бензин,1982.0,1.8 MT (90 л.с.),4.0,1986,SEDAN MECHANICAL 1.8,MECHANICAL,...,LEFT,NaN,3.0,ORIGINAL,True,NaN,60000.0,2020-09-06T06:49:40Z,NaN,100
2,Универсал 5 дв.,AUDI,0000CC,бензин,1988.0,2.3 MT (136 л.с.) 4WD,5.0,1989,WAGON_5_DOORS MECHANICAL 2.3,MECHANICAL,...,LEFT,NaN,3.0,ORIGINAL,True,NaN,99000.0,2020-09-02T14:04:21Z,NaN,100
3,Седан,AUDI,CACECB,бензин,1988.0,1.8 MT (90 л.с.),4.0,1989,SEDAN MECHANICAL 1.8,MECHANICAL,...,LEFT,NaN,3.0,ORIGINAL,True,NaN,65000.0,2020-08-23T17:40:09Z,NaN,100
4,Седан,AUDI,040001,бензин,1990.0,2.0 MT (101 л.с.),4.0,1991,SEDAN MECHANICAL 2.0,MECHANICAL,...,LEFT,NaN,3.0,DUPLICATE,True,NaN,100000.0,2020-09-08T09:22:07Z,NaN,100


In [22]:
len(train)

89378

In [23]:
len(train[train['price']>3_000_000])

9051

In [24]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89378 entries, 0 to 89377
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              89377 non-null  object 
 1   brand                 89378 non-null  object 
 2   color                 89378 non-null  object 
 3   fuelType              89378 non-null  object 
 4   modelDate             89377 non-null  float64
 5   name                  89377 non-null  object 
 6   numberOfDoors         89377 non-null  float64
 7   productionDate        89378 non-null  int64  
 8   vehicleConfiguration  89377 non-null  object 
 9   vehicleTransmission   89377 non-null  object 
 10  engineDisplacement    89377 non-null  object 
 11  enginePower           89377 non-null  float64
 12  description           86124 non-null  object 
 13  mileage               89378 non-null  int64  
 14  Комплектация          89378 non-null  object 
 15  Привод             

In [25]:
test.head(5)

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2 LTR,105 N12,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,...,LIFTBACK ROBOT 1.2,роботизированная,EUROPEAN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,чёрный,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6 LTR,110 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,LIFTBACK MECHANICAL 1.6,механическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
2,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/superb/11...,серый,"{""id"":""20026336"",""name"":""Ambition"",""available_...","Все автомобили, представленные в продаже, прох...",1.8 LTR,152 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,...,LIFTBACK ROBOT 1.8,роботизированная,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
3,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,коричневый,"{""id"":""20803582"",""name"":""Ambition"",""available_...",КОМПЛЕКТ ЗИМНЕЙ (ЛЕТНЕЙ) РЕЗИНЫ ПО СЕЗОНУ В ПО...,1.6 LTR,110 N12,"{""cruise-control"":true,""roller-blind-for-rear-...",бензин,...,LIFTBACK AUTOMATIC 1.6,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
4,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,белый,NaN,ЛОТ: 01220889\nАвтопрага Север\n\nВы можете по...,1.8 LTR,152 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,LIFTBACK AUTOMATIC 1.8,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен


In [26]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   bodyType              34686 non-null  object
 1   brand                 34686 non-null  object
 2   car_url               34686 non-null  object
 3   color                 34686 non-null  object
 4   complectation_dict    6418 non-null   object
 5   description           34686 non-null  object
 6   engineDisplacement    34686 non-null  object
 7   enginePower           34686 non-null  object
 8   equipment_dict        24690 non-null  object
 9   fuelType              34686 non-null  object
 10  image                 34686 non-null  object
 11  mileage               34686 non-null  int64 
 12  modelDate             34686 non-null  int64 
 13  model_info            34686 non-null  object
 14  model_name            34686 non-null  object
 15  name                  34686 non-null

In [27]:
test.drop(['image'], axis=1,inplace=True)

## Data Preprocessing

In [28]:
# ... 

In [29]:
train.dropna(subset=['productionDate','mileage'], inplace=True)
train.dropna(subset=['price'], inplace=True)

In [30]:
# сформируем списки столбцов по группам исходя из типов признаков
num_cols = ['mileage', 'modelDate', 'productionDate']
#bin_cols =['Руль', 'Состояние', 'Таможня', 'ПТС'] 
bin_cols =['Руль', 'Состояние', 'ПТС'] 
category_cols = ['model_info', 'model_name', 'name', 'super_gen', 'complectation_dict', 'equipment_dict',  'bodyType', 'brand', 'color', 'fuelType', 'numberOfDoors', 
                 'priceCurrency',  'vehicleConfiguration', 'vehicleTransmission', 'vendor', 'Владельцы', 'Владение', 'Привод', 'engineDisplacement', 'enginePower']
# 'priceCurrency' - тип валюты. В настоящее время у нас только рубли, но могут быть и др. варианты. Поэтому отнесем к категориальным данным 

In [31]:
# для baseline просто возьму пару схожих признаков без полной обработки
#columns = ['bodyType', 'brand', 'productionDate', 'engineDisplacement', 'mileage','Владельцы']
y = train['price']
train.drop(['price'], axis=1,inplace=True)
df_train = train#[columns]
df_test = test#[columns]

## Label Encoding

In [32]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [33]:
#for colum in ['bodyType', 'brand', 'engineDisplacement']:
#    data[colum] = data[colum].astype('category').cat.codes
for colum in category_cols:
    data[colum] = data[colum].astype('category').cat.codes

In [34]:
#data['Таможня'].value_counts()

In [35]:
data["Состояние"] = data["Состояние"].fillna('Неизвестно')
#data["Таможня"] = data["Таможня"].fillna('Растаможен')
data["ПТС"] = data["ПТС"].fillna('Неизвестно')

#data[bin_cols]

In [36]:
#data["Владельцы"] = data["Владельцы"].str.replace(u'\xa0', '')
#label_encoder = LabelEncoder()
#for column in columns:
#data['Владельцы'] = label_encoder.fit_transform(data['Владельцы'])
label_encoder = LabelEncoder()

for column in bin_cols:
    #print(column)
    data[column] = label_encoder.fit_transform(data[column])
    
# убедимся в преобразовании    
data.head()

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,ПТС,Привод,Руль,Состояние,Таможня,sample,Комплектация,start_date,hidden,model
0,176,29,https://auto.ru/cars/used/sale/skoda/octavia/1...,29,-1,"Все автомобили, представленные в продаже, прох...",11,401,18865,0,...,4,1,2,0,Растаможен,0,NaN,NaN,NaN,NaN
1,176,29,https://auto.ru/cars/used/sale/skoda/octavia/1...,31,-1,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,27,406,5438,0,...,4,1,2,0,Растаможен,0,NaN,NaN,NaN,NaN
2,176,29,https://auto.ru/cars/used/sale/skoda/superb/11...,28,30,"Все автомобили, представленные в продаже, прох...",33,444,12212,0,...,4,1,2,0,Растаможен,0,NaN,NaN,NaN,NaN
3,176,29,https://auto.ru/cars/used/sale/skoda/octavia/1...,22,599,КОМПЛЕКТ ЗИМНЕЙ (ЛЕТНЕЙ) РЕЗИНЫ ПО СЕЗОНУ В ПО...,27,406,11781,0,...,4,1,2,0,Растаможен,0,NaN,NaN,NaN,NaN
4,176,29,https://auto.ru/cars/used/sale/skoda/octavia/1...,17,-1,ЛОТ: 01220889\nАвтопрага Север\n\nВы можете по...,33,444,5433,0,...,4,1,2,0,Растаможен,0,NaN,NaN,NaN,NaN


In [37]:
#Удалим столбцы не несущие в себе параметры автомобиля, и не влиящие на предсказание цены, а так же те которые не успел обработать  
data.drop(['car_url', 'parsing_unixtime', 'sell_id','Комплектация','Таможня','hidden','start_date','model'], axis=1,inplace=True)
# В связи с отсутствие времени на обработку текста, также удалим 'description'
data.drop(['description'], axis=1,inplace=True)
data

,bodyType,brand,color,complectation_dict,engineDisplacement,enginePower,equipment_dict,fuelType,mileage,modelDate,...,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,sample
0,176,29,29,-1,11,401,18865,0,74000,2013.0,...,1097,7,0,5,-1,4,1,2,0,0
1,176,29,31,-1,27,406,5438,0,60563,2017.0,...,1081,6,0,3,-1,4,1,2,0,0
2,176,29,28,30,33,444,12212,0,88000,2013.0,...,1099,7,0,3,-1,4,1,2,0,0
3,176,29,22,599,27,406,11781,0,95000,2013.0,...,1045,4,0,3,-1,4,1,2,0,0
4,176,29,17,-1,33,444,5433,0,58536,2008.0,...,1046,4,0,3,-1,4,1,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123649,57,30,2,-1,97,126,-1,3,260000,2004.0,...,1162,0,-1,2,-1,1,2,0,1,1
123650,57,30,9,-1,97,126,-1,3,300000,2004.0,...,1162,0,-1,2,524,1,2,0,1,1
123651,57,30,2,-1,72,112,-1,3,126000,2013.0,...,1152,0,-1,0,498,1,2,0,1,1
123652,57,30,7,-1,72,112,-1,3,84000,2013.0,...,1152,0,-1,2,520,1,2,0,1,1


In [38]:
cols ={"complectation_dict",'equipment_dict', 'Владение'}
for col in cols:
    data[f'{col}_isNAN'] = pd.isna(data[col]).astype('uint8')

In [39]:
X = data.query('sample == 1').drop(['sample'], axis=1)
X_sub = data.query('sample == 0').drop(['sample'], axis=1)

## Train Split

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

# Model 1: Создадим "наивную" модель 
Эта модель будет предсказывать среднюю цену по модели двигателя (engineDisplacement). 
C ней будем сравнивать другие модели.




In [41]:
tmp_train = X_train.copy()
tmp_train['price'] = y_train

In [42]:
# Находим median по экземплярам engineDisplacement в трейне и размечаем тест
predict = X_test['engineDisplacement'].map(tmp_train.groupby('engineDisplacement')['price'].median())

#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(mape(y_test, predict.values))*100:0.2f}%")

Точность наивной модели по метрике MAPE: 117.95%


# # Model 2 : CatBoost
![](https://pbs.twimg.com/media/DP-jUCyXcAArRTo.png:large)   


У нас в данных практически все признаки категориальные. Специально для работы с такими данными была создана очень удобная библиотека CatBoost от Яндекса. [https://catboost.ai](http://)     
На данный момент **CatBoost является одной из лучших библиотек для табличных данных!**

#### Полезные видео о CatBoost (на русском):
* [Доклад про CatBoost](https://youtu.be/9ZrfErvm97M)
* [Свежий Туториал от команды CatBoost (практическая часть)](https://youtu.be/wQt4kgAOgV0) 

## Fit

In [43]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, y_train,
         #cat_features=cat_features_ids,
         eval_set=(X_test, y_test),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

model.save_model('catboost_single_model_baseline.model')

In [44]:
# оцениваем точность
predict = model.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

Точность модели по метрике MAPE: 15.11%


Вот так просто со старта, даже не трогая сами данные и не подбирая настройки catboosta, получаем модель с уровнем ошибки в 18%!

### Log Traget
Попробуем взять таргет в логорифм - это позволит уменьшить влияние выбросов на обучение модели (используем для этого np.log и np.exp).    
В принциепе мы можем использовать любое приобразование на целевую переменную. Например деление на курс доллара, евро или гречки :) в дату сбора данных, смотрим дату парсинга в тесте в **parsing_unixtime**

In [45]:
np.log(y_train)

53199    13.060488
75086    14.762874
43113    12.762827
69668    12.785491
70499    11.918391
           ...    
6291     14.508657
55136    12.468437
77178    13.217674
865      12.641097
15861    14.403297
Name: price, Length: 71174, dtype: float64

In [46]:
# Поиск наилучших параметров для CatBoostRegressor
cbr = CatBoostRegressor()

grid = {'learning_rate': [round(x,3) for x in np.linspace(start = 0.025, stop = 0.4, num = 40)],
        'depth': [int(x) for x in np.linspace(start = 5, stop = 10, num = 5)],
        'l2_leaf_reg': [int(x) for x in np.linspace(start = 1, stop = 5, num = 5)]}

#grid_search_result = cbr.grid_search(grid, 
#                                       X=X_train, 
#                                       y=y_train, 
#                                       plot=False)
#print(grid_search_result['params'])

# {'depth': 10,'l2_leaf_reg': 2, 'learning_rate': 0.381} bestTest = 543125.899    bestIteration = 331

In [47]:
model = CatBoostRegressor(iterations = 330,
                          learning_rate = 0.381,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          depth = 10,
                          l2_leaf_reg = 2)


model.fit(X_train, np.log(y_train),
         #cat_features=cat_features_ids,
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

#model.save_model('catboost_single_model_2_baseline.model')

In [48]:
predict_test = np.exp(model.predict(X_test))
#predict_submission = np.round((np.exp(model.predict(X_sub))-1)/1000) *1000
predict_submission = np.exp(model.predict(X_sub))

In [49]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Точность модели по метрике MAPE: 12.98%


Как видим точность возросла до 13%, а что будет на ЛБ?

# Submission

In [50]:
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_2_v{VERSION}.csv', index=False)
#sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

,sell_id,price
0,1100575026,1.780288e+06
1,1100549428,2.220909e+06
2,1100658222,2.197047e+06
3,1100937408,1.822631e+06
4,1101037972,1.878359e+06
5,1100912634,1.853629e+06
6,1101228730,1.609548e+06
7,1100165896,1.322105e+06
8,1100768262,3.678947e+06
9,1101218501,3.208966e+06


В итоге получили **MAPE 27%** на ЛБ!

Большая разница в ошибке может указывать на то что тест и трейн имеют различия по выборке или то что данные в трейне могли уже устареть и их нужно обновлять.

# What's next?
Или что еще можно сделать, чтоб улучшить результат:

* Спарсить свежие данные 
* Посмотреть, что можно извлечь из признаков или как еще можно обработать признаки
* Сгенерировать новые признаки
* Попробовать подобрать параметры модели
* Попробовать другие алгоритмы и библиотеки ML
* Сделать Ансамбль моделей, Blending, Stacking

Подробный чек лист: https://docs.google.com/spreadsheets/d/1I_ErM3U0Cs7Rs1obyZbIEGtVn-H47pHNCi4xdDgUmXY/edit?usp=sharing